In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

%run helpFunction.ipynb

\begin{table}
\caption{Test Klassifikations Metriken}
\label{tab:model_performance}
\begin{tabular}{llllr}
\toprule
Modell & Accuracy & Precision & Recall & F1-Score \\
\midrule
ViT & 83% & 83% & 83% & 83.000000 \\
Hybridmodel - ViT - DeiT & 82% & 82% & 82% & 82.000000 \\
Hybridmodel - ViT Lite - DeiT & 82% & 82% & 82% & 82.000000 \\
InceptionV3 & 82% & 83% & 82% & 82.000000 \\
DeiT & 81% & 81% & 82% & 81.000000 \\
ViT Lite & 79% & 79% & 79% & 79.000000 \\
Basic CNN & 78% & 80% & 78% & 78.000000 \\
\bottomrule
\end{tabular}
\end{table}



/var/folders/rg/wpq9g3gn2_sg42m9vzpp7flw0000gn/T/ipykernel_9935/671559329.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Modell').apply(lambda x: x.nlargest(1, 'F1-Score')).reset_index(drop=True)


In [2]:
import sys
sys.version

'3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]'

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [4]:
wandb.init(project="VisionTransformer")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: silvan-wiedmer (silvan-wiedmer-fhgr). Use `wandb login --relogin` to force relogin


In [5]:
# Konfiguration
image_size = (250, 250)
batch_size = 32

In [6]:
path_with_sign = 'data/y'
path_without_sign = 'data/n'

In [7]:
def load_images_from_folder(folder, label, target_size):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Fehler beim Laden des Bildes {img_path}: {e}")
    return np.array(images), np.array(labels)

In [8]:
images_with_sign, labels_with_sign = load_images_from_folder(path_with_sign, 1, image_size)
images_without_sign, labels_without_sign = load_images_from_folder(path_without_sign, 0, image_size)

In [9]:
all_images = np.concatenate((images_with_sign, images_without_sign), axis=0)
all_labels = np.concatenate((labels_with_sign, labels_without_sign), axis=0)

In [10]:
all_images /= 255.0
all_images

array([[[[0.32941177, 0.35686275, 0.29411766],
         [0.3254902 , 0.3529412 , 0.28235295],
         [0.22745098, 0.27450982, 0.18039216],
         ...,
         [0.60784316, 0.6156863 , 0.6117647 ],
         [0.63529414, 0.6392157 , 0.64705884],
         [0.6039216 , 0.60784316, 0.6156863 ]],

        [[0.3882353 , 0.42352942, 0.35686275],
         [0.3254902 , 0.36078432, 0.29411766],
         [0.27058825, 0.32156864, 0.21960784],
         ...,
         [0.6       , 0.60784316, 0.6039216 ],
         [0.63529414, 0.6392157 , 0.64705884],
         [0.6431373 , 0.64705884, 0.654902  ]],

        [[0.34509805, 0.3882353 , 0.3254902 ],
         [0.3372549 , 0.3882353 , 0.31764707],
         [0.24705882, 0.29803923, 0.19607843],
         ...,
         [0.627451  , 0.63529414, 0.6313726 ],
         [0.6313726 , 0.63529414, 0.6431373 ],
         [0.6156863 , 0.61960787, 0.627451  ]],

        ...,

        [[0.35686275, 0.3882353 , 0.29803923],
         [0.5019608 , 0.50980395, 0.42352942]

In [11]:
train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)
train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [12]:
train_images.shape, test_images.shape, validation_images.shape

((931, 250, 250, 3), (292, 250, 250, 3), (233, 250, 250, 3))

In [13]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator()

In [14]:
train_generator = train_datagen.flow(train_images, train_labels, batch_size=batch_size)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=batch_size)
validation_generator = test_datagen.flow(validation_images, validation_labels, batch_size=batch_size)

In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


/Users/silvanwiedmer/Desktop/VisionTransformer/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 248, 248, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 59, 59, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    13,779,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,872,449 (52.92 MB)

 Trainable params: 13,872,449 (52.92 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=len(validation_images) // batch_size,
    callbacks=[WandbMetricsLogger()]
)

Epoch 1/100


/Users/silvanwiedmer/Desktop/VisionTransformer/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 14s 407ms/step - accuracy: 0.5058 - loss: 1.4420 - val_accuracy: 0.5268 - val_loss: 0.6825
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5312 - loss: 0.6669 - val_accuracy: 0.3333 - val_loss: 0.7368
Epoch 3/100


2024-09-26 20:10:27.941254: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-09-26 20:10:27.982859: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 411ms/step - accuracy: 0.5553 - loss: 0.6783 - val_accuracy: 0.5402 - val_loss: 0.7217
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3438 - loss: 0.8218 - val_accuracy: 0.3333 - val_loss: 0.7021
Epoch 5/100


...

2024-09-26 20:11:08.758239: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 407ms/step - accuracy: 0.5415 - loss: 0.6662 - val_accuracy: 0.6205 - val_loss: 0.6374
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7188 - loss: 0.6466 - val_accuracy: 0.6667 - val_loss: 0.5760
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 424ms/step - accuracy: 0.6432 - loss: 0.6463 - val_accuracy: 0.6250 - val_loss: 0.6129
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7812 - loss: 0.5029 - val_accuracy: 0.6667 - val_loss: 0.7071
Epoch 13/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 415ms/step - accuracy: 0.6561 - loss: 0.6282 - val_accuracy: 0.4688 - val_loss: 0.7311
Epoch 14/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5312 - loss: 0.6891 - val_accuracy: 0.5556 - val_loss: 0.7113
Epoch 15/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 426ms/step - accuracy: 0.5838 - loss: 0.6544 - val_accuracy: 0.6384 - val_loss: 0.6066
Epoch 16/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7188 - loss: 0.5836 - val_accuracy: 0.6

2024-09-26 20:12:03.093216: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 409ms/step - accuracy: 0.6915 - loss: 0.5981 - val_accuracy: 0.6295 - val_loss: 0.6155
Epoch 18/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6250 - loss: 0.6408 - val_accuracy: 0.4444 - val_loss: 0.6857
Epoch 19/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 401ms/step - accuracy: 0.7132 - loss: 0.5708 - val_accuracy: 0.4955 - val_loss: 0.7149
Epoch 20/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5625 - loss: 0.6503 - val_accuracy: 0.5556 - val_loss: 0.6978
Epoch 21/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 421ms/step - accuracy: 0.6672 - loss: 0.6064 - val_accuracy: 0.6429 - val_loss: 0.6331
Epoch 22/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5938 - loss: 0.6440 - val_accuracy: 0.5556 - val_loss: 0.6269
Epoch 23/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 419ms/step - accuracy: 0.6761 - loss: 0.6125 - val_accuracy: 0.6830 - val_loss: 0.5760
Epoch 24/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 0.5153 - val_accuracy: 0.5

KeyboardInterrupt: 

In [ ]:
# Modell evaluieren
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Testgenauigkeit: {test_acc:.4f}, Testverlust: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.suptitle('Model Training - Basic CNN')
plt.tight_layout()
plt.show()


In [ ]:
predictions = model.predict(test_images)
predictions = (predictions > 0.5)

prediction_csv(test_labels, predictions, 'Basic CNN')
print(classification_report(test_labels, predictions))

In [ ]:
namelabels = {0: 'Wartelinie', 1: 'keine Wartelinie'}
cm = confusion_matrix(test_labels, predictions)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=namelabels.values(), yticklabels=namelabels.values())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - Basic CNN Model')
plt.show()

In [ ]:
wandb.finish()